<a href="https://colab.research.google.com/github/tys203831/bursa-scraper/blob/main/notebook/Bursa_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bursa Stock Scraper 

[Demo App here](https://tys203831-bursa-scraper-bursa-scraper-1okr1g.streamlitapp.com/)

## Input

In [1]:
rf: float = 4.00 # risk free rate (in percentage %) 
period: int = 5 # e.g. 5 = 5 years
interval: str = "1 week" # options = "1 day", "1 week", "1 month", 3 month""
confidence_level: float = 1.0 # confidence_level (in percentage %)
include_dividends: bool = True # include dividends into stock return calculation

## Setup

In [2]:
# no need amend, haven't worked perfectly as they should be 
skipna: bool = False # skip NaN values for annualized excess return & annualized standard deviation
exclude_warrant: bool = True

interval_dict = dict(zip(["1 day","1 week","1 month","3 month"], ["1d","1wk","1mo","3mo"]))
interval = interval_dict[interval]
confidence_level = confidence_level/100
rf = rf/100

In [3]:
pip install requests requests_html yfinance gspread statsmodels scipy bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Step 1: connect to google spreadsheet
import google.auth
from google.colab import auth
import gspread

auth.authenticate_user()
credentials, project_id = google.auth.default()

gc = gspread.authorize(credentials)

gs = gc.open_by_url("https://docs.google.com/spreadsheets/d/1mJjBNTGAF3CLOi1aeJDUSWx6EvtRqB-RhJdvnBh1iAU/edit#gid=14691771")

## Retrieve Data & Data Cleaning

### (i) Get stock ticker from Bursa

In [5]:
# Step 4: Get stock tickers from i3investor and then upload to the google spreadsheet: "Bursa stock list" sheet
# start scraping stock list from i3investor
import pandas
import requests
import json

url = "https://klse.i3investor.com/wapi/web/stock/listing/datatables"
data = {"dtDraw":1,"start":0,"order":[{"column":1,"dir":"asc"}],"page":0,"size":100,"marketList":[],"sectorList":[],"subsectorList":[],"type":"","stockType":""}
retries = 2

session = requests.Session()
response = session.post(url, json=data)

df_stock_list = pandas.DataFrame()

#for idx in range(0,11,1):
while len(response.json()["data"]) != 0:
  try:
    response = session.post(url, json=data)
  
  except requests.exceptions.HTTPError as err:
    code = err.response.status_code

    if code in [429,500,502,503,504]:
      continue
    
    else:
      for retry in range(0,retries,1):
        response = session.post(url, json=data)
        print(response.raise_for_status())
    
  finally:
    df_stock_list= pandas.DataFrame(json.loads(response.text)["data"]) if df_stock_list.empty else pandas.concat((df_stock_list, pandas.DataFrame(json.loads(response.text)["data"])), axis=0)
    #workSheet.append_rows(df_stock_list_values, value_input_option='RAW', table_range ="A1")

    data["dtDraw"] +=1
    data["start"] +=100
    data["page"]+=1

# data cleaning for this stock ticker table, e.g., drop unwanted columns and reordering them
clean_df_stock_list = df_stock_list.drop([0,1,8,11], axis=1) # drop unwanted columns
clean_df_stock_list = clean_df_stock_list[[12,13,10,14,9, 2,3,4,5,6,7]] # reordering the columns
clean_df_stock_list.columns = ["STOCK_SYMBOL","STOCK CODE","SECTOR","SUBSECTOR", "MKT", "OPEN","LAST","CHG%","CHG","VOL","MKT CAP"] #rename the columns

clean_df_stock_list[["OPEN","LAST","CHG%","CHG","VOL","MKT CAP"]] \
= clean_df_stock_list[["OPEN","LAST","CHG%","CHG","VOL","MKT CAP"]].replace(
            {"<((\s|\w|[='-/])+)>":"",
                         "K": "E+03",
                         "M": "E+06",
                        "B": "E+09",}, regex=True)

clean_df_stock_list["MKT CAP"] = clean_df_stock_list["MKT CAP"].str.replace(" ","") #.map(pandas.eval).astype(float) # convert string (e.g., 897 M)
clean_df_stock_list["VOL"] = clean_df_stock_list["VOL"].str.replace(" ","") #.map(pandas.eval).astype(int)
clean_df_stock_list= clean_df_stock_list.apply(pandas.to_numeric, errors="ignore") 
print(clean_df_stock_list.dtypes)
clean_df_stock_list.sort_values("MKT CAP",ascending=False)


STOCK_SYMBOL     object
STOCK CODE       object
SECTOR           object
SUBSECTOR        object
MKT              object
OPEN            float64
LAST            float64
CHG%            float64
CHG             float64
VOL             float64
MKT CAP         float64
dtype: object


,STOCK_SYMBOL,STOCK CODE,SECTOR,SUBSECTOR,MKT,OPEN,LAST,CHG%,CHG,VOL,MKT CAP
61,MAYBANK,1155,FINANCIAL SERVICES,BANKING,MAIN,8.970,8.950,-0.22,-0.020,384000.0,1.071316e+11
14,PBBANK,1295,FINANCIAL SERVICES,BANKING,MAIN,4.650,4.640,-0.22,-0.010,1202000.0,9.006561e+10
16,PCHEM,5183,INDUSTRIAL PRODUCTS & SERVICES,CHEMICALS,MAIN,8.820,8.850,0.57,0.050,15000.0,7.080000e+10
72,CIMB,1023,FINANCIAL SERVICES,BANKING,MAIN,5.430,5.420,-0.18,-0.010,239000.0,5.677048e+10
15,IHH,5225,HEALTH CARE,HEALTH CARE PROVIDERS,MAIN,6.490,6.440,-0.77,-0.050,42000.0,5.671092e+10
...,...,...,...,...,...,...,...,...,...,...,...
89,HSI-HEK,0651EK,,,,0.165,0.165,3.13,0.005,100000.0,0.000000e+00
88,HSI-CJE,0651JE,,,,0.000,0.035,0.00,0.000,0.0,0.000000e+00
22,AGMO,0258,,,,NaN,NaN,NaN,NaN,NaN,0.000000e+00
33,ALIBABA-C12,058212,,,,0.000,0.255,0.00,0.000,0.0,0.000000e+00


In [6]:
# data cleaning for this stock ticker table, e.g., drop unwanted columns and reordering them
clean_df_stock_list = df_stock_list.drop([0,1,8,11], axis=1) # drop unwanted columns
clean_df_stock_list = clean_df_stock_list[[12,13,10,14,9, 2,3,4,5,6,7]] # reordering the columns
clean_df_stock_list.columns = ["STOCK_SYMBOL","STOCK CODE","SECTOR","SUBSECTOR", "MKT", "OPEN","LAST","CHG%","CHG","VOL","MKT CAP"] #rename the columns

clean_df_stock_list[["OPEN","LAST","CHG%","CHG","VOL","MKT CAP"]] \
= clean_df_stock_list[["OPEN","LAST","CHG%","CHG","VOL","MKT CAP"]].replace(
            {"<((\s|\w|[='-/])+)>":"",
                         "K": "E+03",
                         "M": "E+06",
                        "B": "E+09",}, regex=True)

clean_df_stock_list["MKT CAP"] = clean_df_stock_list["MKT CAP"].str.replace(" ","") #.map(pandas.eval).astype(float) # convert string (e.g., 897 M)
clean_df_stock_list["VOL"] = clean_df_stock_list["VOL"].str.replace(" ","") #.map(pandas.eval).astype(int)
clean_df_stock_list= clean_df_stock_list.apply(pandas.to_numeric, errors="ignore") 
print(clean_df_stock_list.dtypes)
clean_df_stock_list.sort_values("MKT CAP",ascending=False)

STOCK_SYMBOL     object
STOCK CODE       object
SECTOR           object
SUBSECTOR        object
MKT              object
OPEN            float64
LAST            float64
CHG%            float64
CHG             float64
VOL             float64
MKT CAP         float64
dtype: object


,STOCK_SYMBOL,STOCK CODE,SECTOR,SUBSECTOR,MKT,OPEN,LAST,CHG%,CHG,VOL,MKT CAP
61,MAYBANK,1155,FINANCIAL SERVICES,BANKING,MAIN,8.970,8.950,-0.22,-0.020,384000.0,1.071316e+11
14,PBBANK,1295,FINANCIAL SERVICES,BANKING,MAIN,4.650,4.640,-0.22,-0.010,1202000.0,9.006561e+10
16,PCHEM,5183,INDUSTRIAL PRODUCTS & SERVICES,CHEMICALS,MAIN,8.820,8.850,0.57,0.050,15000.0,7.080000e+10
72,CIMB,1023,FINANCIAL SERVICES,BANKING,MAIN,5.430,5.420,-0.18,-0.010,239000.0,5.677048e+10
15,IHH,5225,HEALTH CARE,HEALTH CARE PROVIDERS,MAIN,6.490,6.440,-0.77,-0.050,42000.0,5.671092e+10
...,...,...,...,...,...,...,...,...,...,...,...
89,HSI-HEK,0651EK,,,,0.165,0.165,3.13,0.005,100000.0,0.000000e+00
88,HSI-CJE,0651JE,,,,0.000,0.035,0.00,0.000,0.0,0.000000e+00
22,AGMO,0258,,,,NaN,NaN,NaN,NaN,NaN,0.000000e+00
33,ALIBABA-C12,058212,,,,0.000,0.255,0.00,0.000,0.0,0.000000e+00


In [7]:
float('0.151 E+06'.replace(" ", ""))

151000.0

In [8]:
# update stock code list that need to scrape

import re


stock_list = clean_df_stock_list["STOCK CODE"]
full_stock_list = ["^KLSE"]
full_stock_list.extend(stock_list)

if exclude_warrant: 
  full_stock_list = [stock for stock in full_stock_list if not bool(re.match(pattern="\d+[a-zA-Z]+",string=stock))]

print(full_stock_list)

['^KLSE', '03041', '0012', '7054', '5238', '7167', '7086', '2488', '03028', '7131', '0218', '5281', '7191', '9148', '7146', '0181', '6599', '5139', '5185', '5198', '7145', '0258', '7315', '7090', '0122', '0209', '5014', '2658', '7609', '5115', '5116', '2674', '058212', '1163', '0079', '5269', '1015', '5293', '0159', '5120', '03011', '7031', '6351', '7083', '0048', '4758', '0226', '6556', '5082', '9342', '03051', '5568', '5088', '5015', '6432', '0119', '7214', '7181', '7007', '5210', '5127', '0038', '1481', '0068', '7722', '7129', '4057', '0105', '7162', '03032', '6399', '0072', '8176', '5302', '7048', '5130', '7099', '03037', '8885', '5204', '7579', '6888', '5106', '7120', '2305', '5021', '7078', '03012', '0098', '7251', '4162', '5248', '6602', '0187', '6173', '5190', '9814', '7668', '8133', '7005', '5258', '0195', '6998', '0179', '5032', '3239', '3395', '5196', '4219', '1899', '5069', '0168', '9288', '7036', '6297', '5254', '5100', '5932', '9938', '7221', '2771', '0011', '7188', '1818

### (ii) Get stock & dividend

*   stock price, stock return
*   dividend

#### -- Get stock price history

In [9]:
# Step 2: create a function to get stock & index price history from yahoo finance
re
import pandas
import yfinance
from typing import Union

def getData(ticker_code: Union[str, list], period:str, interval:str) -> pandas.DataFrame:
  # get stock return dataframe of the Bursa stocks
  if isinstance(ticker_code, list):
    ticker_code = [str(ticker) + ".KL" if not str(ticker).endswith(".KL") and not ticker.startswith("^") else str(ticker) for ticker in ticker_code]
    ticker_code = " ".join(ticker_code)
  elif isinstance(ticker_code, str) and not ticker_code.endswith(".KL") and not ticker_code.startswith("^"): 
    ticker_code = ticker_code + ".KL"
  df_stock = yfinance.download(tickers=ticker_code, period=period, interval=interval, group_by="column")
  df_stock = pandas.DataFrame(df_stock["Adj Close"]).fillna(method="ffill", axis=0)
  return df_stock.reset_index()

def getReturn(df_stock: pandas.DataFrame) -> pandas.DataFrame:
  df_stock["Date"] = pandas.to_datetime(df_stock["Date"])
  df_stock_return = df_stock.set_index("Date").pct_change()
  return df_stock_return[1:].reset_index()

import dateutil
import datetime

def filterDataBasedYear(df, period: int):
  # get stock history of previous n years (e.g. 2 years stock return data out of 5 years)
  n_years_from_today = datetime.date.today() - dateutil.relativedelta.relativedelta(years=+period)
  n_years_from_today = n_years_from_today.strftime("%Y-%m-%d")
  return df[df["Date"]>= n_years_from_today]

In [10]:
# Step 5: Download stock return dataframe of Bursa Malaysia stocks from yahoo finance using yfinance
#stock_list = workSheet.col_values(2)[1:]

stock_df = getData(ticker_code=full_stock_list, period=str(period)+"y", interval=interval)
stock_df 

[*********************100%***********************]  990 of 990 completed

9 Failed downloads:
- 058212.KL: No data found, symbol may be delisted
- 0259.KL: No data found, symbol may be delisted
- 0258.KL: No data found, symbol may be delisted
- 054810.KL: No data found, symbol may be delisted
- 054620.KL: No data found, symbol may be delisted
- 3055.KL: No data found, symbol may be delisted
- 0045.KL: No data found, symbol may be delisted
- 058312.KL: No data found, symbol may be delisted
- 03009.KL: No data found, symbol may be delisted


,Date,0001.KL,0002.KL,0005.KL,0006.KL,0007.KL,0008.KL,0010.KL,0011.KL,0012.KL,...,9814.KL,9822.KL,9873.KL,9881.KL,9938.KL,9946.KL,9954.KL,9962.KL,9997.KL,^KLSE
0,2017-08-14,0.171756,1.638836,0.08,0.35,0.650,0.617109,0.160,0.251474,1.201192,...,0.460,1.456134,0.628316,0.560077,0.256,0.427612,0.057037,0.377026,0.631801,1776.219971
1,2017-08-15,0.171756,1.638836,0.08,0.35,0.650,0.617109,0.160,0.251474,1.201192,...,0.460,1.456134,0.628316,0.560077,0.256,0.427612,0.057037,0.377026,0.631801,1776.219971
2,2017-08-16,0.171756,1.638836,0.08,0.35,0.650,0.617109,0.160,0.251474,1.201192,...,0.460,1.456134,0.628316,0.560077,0.256,0.427612,0.057037,0.377026,0.631801,1776.219971
3,2017-08-17,0.171756,1.638836,0.08,0.35,0.650,0.617109,0.160,0.251474,1.201192,...,0.460,1.456134,0.628316,0.560077,0.256,0.427612,0.057037,0.377026,0.631801,1776.219971
4,2017-08-18,0.171756,1.638836,0.08,0.35,0.650,0.617109,0.160,0.251474,1.201192,...,0.460,1.456134,0.628316,0.560077,0.256,0.427612,0.057037,0.377026,0.631801,1776.219971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,2022-08-08,1.370000,4.090000,0.09,1.01,0.025,0.370000,0.135,0.285000,0.860000,...,0.085,4.460000,0.500000,0.570000,0.185,0.160000,0.375000,0.285000,0.570000,1506.189941
1132,2022-08-09,1.370000,4.090000,0.09,1.01,0.025,0.370000,0.135,0.285000,0.860000,...,0.085,4.460000,0.500000,0.570000,0.185,0.160000,0.375000,0.285000,0.570000,1506.189941
1133,2022-08-11,1.370000,4.090000,0.09,1.01,0.025,0.370000,0.135,0.285000,0.860000,...,0.085,4.460000,0.500000,0.570000,0.185,0.160000,0.375000,0.285000,0.570000,1506.189941
1134,2022-08-12,1.370000,4.090000,0.09,1.01,0.025,0.370000,0.135,0.285000,0.860000,...,0.085,4.460000,0.500000,0.570000,0.185,0.160000,0.375000,0.285000,0.570000,1506.189941


#### -- Get dividend history

In [11]:
# Step 6: Download dividend dataframe of Bursa Malaysia stocks from yahoo finance using yfinance

#stock_list = workSheet.col_values(2)[1:]
import yfinance
import pandas 
import threading
from typing import Union, List, Dict
import datetime
from queue import Queue

multilevel_dict = {}
threads = []

def bursa_ticker_amend(ticker_code: Union[list, str]) -> List[str]:
  if isinstance(ticker_code, list):
    ticker_code = [str(ticker) + ".KL" if not str(ticker).endswith(".KL") and not ticker.startswith("^") else str(ticker) for ticker in ticker_code]
  elif isinstance(ticker_code, str) and not ticker_code.endswith(".KL") and not ticker_code.startswith("^"): 
    ticker_code = ticker_code + ".KL"
  return ticker_code

# scrape dividend history from yahoo finance
def dividend_scraper(stock:str) -> Dict[str, Dict[datetime.datetime ,float]]:  
  try:
    ticker = yfinance.Ticker(stock)
    dividends = ticker.get_dividends().to_dict()
    multilevel_dict.update({stock:dividends})   
  except AttributeError:
    print("No data found! No dividends are distributed!")
  return multilevel_dict

full_bursa_stock_list = bursa_ticker_amend(full_stock_list)

if include_dividends:
  # Create 4 scraper workers
  for idx in range(len(full_bursa_stock_list)): # range(0, len(full_stock_list),1):
    t =threading.Thread(target=dividend_scraper,args=(full_bursa_stock_list[idx],))
    t.start()
    threads.append(t)

for thread in threads:
  thread.join()

- 0258.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 058212.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 054810.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 3055.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 0045.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 058312.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 03009.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 0259.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!
- 054620.KL: No data found, symbol may be delisted
No data found! No dividends are distributed!


#### -- Merge dataframe of stock price & dividend history

In [12]:
if include_dividends:
  dividend_history = pandas.DataFrame.from_dict(multilevel_dict)
  dividend_history = dividend_history.reset_index()
  dividend_history = dividend_history.rename(columns= {"index":"Date"})

  # create a date range series
  datetable = pandas.date_range(start=min(stock_df["Date"]), end = max(stock_df["Date"]), freq="D").to_series(name="Date")

  # merge two dataframes: datetable and dividend_history
  sorted_dividend_history = pandas.merge(datetable, dividend_history, on="Date", how="left")

  new_sorted_dividend_history= sorted_dividend_history.copy(deep=True).reset_index()
  new_sorted_dividend_history["Date"] = pandas.to_datetime(new_sorted_dividend_history["Date"])
  new_sorted_dividend_history = new_sorted_dividend_history.reindex(columns=stock_df.columns).fillna(0)
  new_sorted_dividend_history

  stock_with_div_df = stock_df.set_index("Date").add(new_sorted_dividend_history.set_index("Date"))
  if interval != "1d":
    resample_period = {"1wk": "W","1mo":"M","3mo": "Q"}
    stock_with_div_df = stock_with_div_df.resample(resample_period[interval]).mean().reset_index()
  else:
    stock_with_div_df = stock_with_div_df.mean().reset_index()

  stock_with_div_df

#### -- Get stock return (including dividend)

In [13]:
if include_dividends:
  total_stock_return_df = getReturn(stock_with_div_df)
  
else:
  total_stock_return_df = getReturn(stock_df)
total_stock_return_df

,Date,0001.KL,0002.KL,0005.KL,0006.KL,0007.KL,0008.KL,0010.KL,0011.KL,0012.KL,...,9814.KL,9822.KL,9873.KL,9881.KL,9938.KL,9946.KL,9954.KL,9962.KL,9997.KL,^KLSE
0,2017-08-27,-0.028571,0.005155,0.812500,0.085714,0.000000,-0.045752,0.093750,0.000000,-0.030075,...,0.000000,-0.030667,0.069231,-0.008475,-0.015625,-0.024390,-0.407421,0.000000,-0.014815,-0.003969
1,2017-09-03,0.000000,-0.005128,0.034483,0.184211,-0.038462,-0.027397,-0.028571,0.109375,-0.015504,...,-0.032609,-0.024759,0.000000,-0.017094,-0.031746,-0.033333,0.000000,-0.011905,-0.015038,0.002255
2,2017-09-10,-0.029412,0.000000,0.566667,-0.077778,0.120000,-0.063380,0.088235,-0.112676,0.007874,...,-0.033708,-0.029619,0.028777,0.121739,0.016393,-0.017241,0.000000,0.000000,-0.068702,0.003801
3,2017-09-17,0.060606,-0.025773,0.000000,-0.012048,0.035714,-0.037594,-0.027027,0.117575,-0.031250,...,0.023256,0.005814,-0.013986,0.108527,0.048387,-0.017544,0.750036,-0.012048,-0.016394,0.003613
4,2017-09-24,-0.028571,-0.031746,-0.085106,-0.146341,0.000000,-0.031250,-0.027778,-0.084274,-0.072581,...,-0.022727,-0.028902,-0.021277,-0.069930,-0.019231,0.004464,0.107138,-0.036585,0.000000,-0.008559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2022-07-24,0.063063,0.012658,0.235294,0.000000,0.000000,0.028169,0.181818,0.036364,0.029762,...,0.000000,0.107738,0.042105,0.096154,-0.027778,-0.150410,0.075758,-0.017544,0.019048,0.033389
257,2022-07-31,0.033898,0.025000,0.000000,0.000000,0.111111,0.000000,-0.038462,0.052632,0.011561,...,0.000000,0.221924,0.060606,0.000000,0.085714,0.086497,-0.056338,0.017857,0.009346,0.018031
258,2022-08-07,-0.008197,0.024390,-0.095238,0.000000,-0.500000,0.000000,0.000000,-0.033333,-0.011429,...,0.000000,0.171504,-0.047619,-0.017544,-0.026316,-0.111111,0.119403,0.000000,-0.018519,0.006246
259,2022-08-14,0.132231,-0.026190,-0.052632,0.000000,0.000000,0.013699,0.080000,-0.017241,-0.005780,...,-0.055556,0.004505,0.039604,0.017857,0.000000,0.000000,0.000000,0.000000,0.075472,0.003090


#### -- Get beta and alpha using Linear Regression

In [14]:
# Step 3: create function to calculate beta using Linear Regression

import numpy
import pandas
from typing import Tuple, Union
import scipy
import logging
import scipy.stats

def calc_linregress_data(excess_stock_return:list, excess_index_return:list, confidence_level: float, nan_policy: str ="propagate") -> Tuple[float, float, float, float, float, str]:
  beta, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(x=excess_index_return, y=excess_stock_return)
  rsquared = rvalue**2
  # skewsness and kurtosis reference: https://www.analyticsvidhya.com/blog/2021/05/shape-of-data-skewness-and-kurtosis/
  skewness = scipy.stats.skew(excess_stock_return, nan_policy=nan_policy) # negative skewed return is preferred -> frequent small returns and less big losses -> Reference: https://corporatefinanceinstitute.com/resources/knowledge/other/negatively-skewed-distribution/#:~:text=Negatively%20Skewed%20Distribution%20in%20Finance&text=Although%20many%20finance%20theories%20and,and%20a%20few%20large%20losses.
  pearson_kurtosis = scipy.stats.kurtosis(excess_stock_return, fisher=False, nan_policy=nan_policy) # Mesokurtic or normal distribution when kurtosis=3 
  fisher_kurtosis = scipy.stats.kurtosis(excess_stock_return, fisher=True, nan_policy=nan_policy) # unlike pearson kurtosis, already exclude 3
  normality = normality_test(excess_stock_return, confidence_level)
  return beta, intercept, rsquared, pvalue, stderr, skewness, pearson_kurtosis, fisher_kurtosis, normality

def normality_test(excess_stock_return:list, confidence_level:float) -> Union[bool, str]:
  # The Kolmogorov-Smirnov test - to test normality of stock returns data
  # null hypothesis: the data sample is normal
  # Youtube link: https://www.youtube.com/watch?v=R-MBFCK3p9Q
  try:  
    k2, p = scipy.stats.kstest(excess_stock_return, scipy.stats.norm.cdf)
    if p > confidence_level: # if p-value greater than confidence level, accept null hypothesis 
      return "normal"
    elif p <= confidence_level: # if p-value lesser than confidence level, reject  null hypothesis
      return "not normal"
  except ValueError as error:
    logging.debug(error)

def getRegression(df: pandas.core.frame.DataFrame, period: int, interval:str, rf: float, confidence_level: float) -> pandas.core.frame.DataFrame:
  interval_dict = {"1d": 252,"1wk":52, "1mo":12, "3mo":4}
  df_ny = filterDataBasedYear(df, period).set_index("Date").sub(rf/interval_dict[interval])
  df_ny = df_ny.apply(lambda x: calc_linregress_data(x.values.tolist(), df_ny["^KLSE"].values.tolist(), confidence_level=confidence_level), axis=0)
  df_ny = df_ny.transpose().reset_index()
  df_ny.columns = ["STOCK CODE",f"BETA_{period}Y", f"INTERCEPT_{period}Y", f"R-SQUARED_{period}Y", f"P-VALUE_{period}Y", f"BETA STANDARD ERROR_{period}Y",f"SKEWNESS_{period}Y" , f"PEARSON KURTOSIS_{period}Y", f"FISHER KURTOSIS_{period}Y",f"NORMALITY TEST_{period}Y" ] 
  df_ny["STOCK CODE"] = df_ny["STOCK CODE"].replace({"[.]KL": ""}, regex=True)
  return df_ny

In [15]:
regression_df = getRegression(total_stock_return_df, period=period, interval=interval, rf=rf, confidence_level=confidence_level)
regression_df.sort_values(f"INTERCEPT_{period}Y", ascending=False)

,STOCK CODE,BETA_5Y,INTERCEPT_5Y,R-SQUARED_5Y,P-VALUE_5Y,BETA STANDARD ERROR_5Y,SKEWNESS_5Y,PEARSON KURTOSIS_5Y,FISHER KURTOSIS_5Y,NORMALITY TEST_5Y
62,0091,-10.673794,0.157221,0.005675,0.225162,8.779112,15.625262,249.648725,246.648725,not normal
381,4464,1.831597,0.033597,0.009803,0.110537,1.143843,1.509822,6.765823,3.765823,not normal
130,0174,2.208459,0.033218,0.005507,0.232183,1.844132,14.461619,225.414656,222.414656,not normal
111,0154,2.072332,0.021696,0.019909,0.022607,0.903477,1.403089,7.201847,4.201847,not normal
75,0104,0.724849,0.020355,0.012213,0.074706,0.405062,3.225538,24.896898,21.896898,not normal
...,...,...,...,...,...,...,...,...,...,...
615,5303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,5305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,5306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618,5308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### -- Calculate Annualized Return and Standard Devation

In [34]:
def getAnnualizedReturn(df:pandas.DataFrame, interval:str, type:str="geometric", skipna:bool=False) -> float:
    """Params:
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    """
    annualized_return: float = None
    annualized_return_dict: dict = {"1d": 252,"1wk":52, "1mo":12, "3mo":4}
    if type == "arithmetic":
        arithmetic_mean: float = df.mean(skipna=skipna)
        annualized_return: float = arithmetic_mean * annualized_return_dict[interval]
    if type == "geometric":
        if skipna:
            geometric_mean: float = df.add(1).apply(lambda x: scipy.stats.gmean(x.dropna(inplace=False)), axis=0).sub(1)
        else:
            geometric_mean: float = df.add(1).apply(scipy.stats.gmean, axis=0).sub(1)
        annualized_return: float = ((1+geometric_mean) ** annualized_return_dict[interval]) -1
    return annualized_return

def getAnnualizedStdDeviation(df: pandas.DataFrame, interval:str, skipna:bool) -> float:
    """Params:
    interval : str
        Valid intervals: 1d,1wk,1mo,3mo
    """
    std_deviation: float = df.std(skipna=skipna)
    annualized_std_dict: dict = {"1d": 252,"1wk":52, "1mo":12, "3mo":4}
    annualized_std: float  = std_deviation * annualized_std_dict[interval]
    return annualized_std

In [35]:
# Step 6: Upload 2Y-Beta, 5Y-Beta and standard deviation of equity to Google Spreadsheet: sheet "Calculated" 

import numpy, pandas
import scipy
import numpy

x_list = []
descriptive_df = pandas.DataFrame()

descriptive_df[f"annualized_standard_deviation_of_equity_{period}Y"] = getAnnualizedStdDeviation(filterDataBasedYear(total_stock_return_df, period =2).set_index("Date"), interval=interval, skipna=skipna)
descriptive_df[f"annualized_excess_return_of_equity_{period}Y"] = getAnnualizedReturn(filterDataBasedYear(total_stock_return_df, period=2).set_index("Date"), interval=interval, skipna=skipna)
descriptive_df = descriptive_df.reset_index()
descriptive_df = descriptive_df.rename(columns={"index":"STOCK CODE"})
descriptive_df["STOCK CODE"] = descriptive_df["STOCK CODE"].replace("[.]KL", "", regex=True)

"""
df = df.replace(numpy.nan, "")

workSheet = gs.worksheet("Calculated")
workSheet.clear()
columns = ["Stock Code", "5Y Beta", "2Y Beta", "2Y Standard Deviation of Equity", "2Y Average Return of Equity"]
workSheet.append_row(columns)
workSheet.format("1:1", {"textFormat":{"bold":"True"}})

workSheet.append_rows(descriptive_df.values.tolist(), value_input_option='RAW', table_range="A1")
"""

descriptive_df.sort_values(f"annualized_excess_return_of_equity_{period}Y", ascending=False)

,STOCK CODE,annualized_standard_deviation_of_equity_5Y,annualized_excess_return_of_equity_5Y
75,0104,7.336216,3.501555
635,5673,7.460981,2.955179
589,5273,4.190381,2.163122
62,0091,187.348208,1.597625
797,7172,4.914024,1.529370
...,...,...,...
614,5302,NaN,NaN
615,5303,NaN,NaN
616,5305,NaN,NaN
617,5306,NaN,NaN


In [28]:
descriptive_df.to_csv("./descriptive_df.csv")

### (iii) Get financial ratios

import requests_html

fov_multilevel_dict = {}
threads = []

def getFinancialOverview(ticker:str):
  url =f"https://www.wsj.com/market-data/quotes/MY/XKLS/{ticker}/financials"
  session = requests_html.HTMLSession()
  try:
    response = session.get(url)
  except requests.exceptions.RequestException as error:
    print(error)
  report_heading = response.html.xpath("//div[@class='mod_headerBox']//h3/text()")
  financials_overview = response.html.xpath("//table[contains(@class,'cr_dataTable')]//span/text()")
  financials_overview = list(filter(lambda x: x != " ", financials_overview))

  # transform list data into dict form 
  value_pair = list(filter(lambda x: financials_overview.index(x)%2 == 1, financials_overview)) # extract odd number from list 'overview'
  key_pair = list(filter(lambda x: financials_overview.index(x)%2 == 0, financials_overview)) # extract even number from list 'overview'
  financials_overview = dict(zip(key_pair, value_pair))
  # further clean the dictionary data through deleting and replacing the key pair of dictionary
  # del financials_overview["EPS "]
  for item in financials_overview:
    if item in ["Sales","Tangible Book Value","Operating Profit","Working Capital","Long Term Liabilities","Capital Expenditure","Capital Expenditure TTM"]:
      financials_overview[item +" Per Share"] = financials_overview.pop(item)
  fov_multilevel_dict[ticker] = financials_overview
  return fov_multilevel_dict

# start scraping a list of stocks' financials overview
threads = []

for idx in range(0, len(full_stock_list),1):
  t = threading.Thread(target=getFinancialOverview,args=[full_stock_list[idx]])
  t.start()
  threads.append(t)

for thread in threads:
  thread.join()

financial_summary = pandas.DataFrame.from_dict(fov_multilevel_dict)
financial_summary

## Merge All DataFrame

In [29]:
# merge dataframes of `clean_df_stock_list`, `regression_df`, `descriptive_df`

from functools import reduce

main_dataframe = [clean_df_stock_list, regression_df, descriptive_df]

merged_df = reduce(lambda left, right: pandas.merge(left, right, on="STOCK CODE", how ="left"), main_dataframe)
merged_df.sort_values(f"INTERCEPT_{period}Y", ascending=False)

,STOCK_SYMBOL,STOCK CODE,SECTOR,SUBSECTOR,MKT,OPEN,LAST,CHG%,CHG,VOL,...,INTERCEPT_5Y,R-SQUARED_5Y,P-VALUE_5Y,BETA STANDARD ERROR_5Y,SKEWNESS_5Y,PEARSON KURTOSIS_5Y,FISHER KURTOSIS_5Y,NORMALITY TEST_5Y,annualized_standard_deviation_of_equity_5Y,annualized_excess_return_of_equity_5Y
736,PGB,0091,ENERGY,"ENERGY INFRASTRUCTURE, EQUIPMENT & SERVICES",MAIN,0.000,0.105,0.00,0.000,0.0,...,0.157221,0.005675,0.225162,8.779112,15.625262,249.648725,246.648725,not normal,187.348208,1.597625
741,PHB,4464,PROPERTY,PROPERTY,MAIN,0.000,0.005,0.00,0.000,0.0,...,0.033597,0.009803,0.110537,1.143843,1.509822,6.765823,3.765823,not normal,14.281307,-0.639442
275,EVD,0174,TECHNOLOGY,SOFTWARE,ACE,0.190,0.190,-2.56,-0.005,20000.0,...,0.033218,0.005507,0.232183,1.844132,14.461619,225.414656,222.414656,not normal,38.679450,0.632062
237,EAH,0154,TECHNOLOGY,DIGITAL SERVICES,ACE,0.010,0.010,0.00,0.000,20000.0,...,0.021696,0.019909,0.022607,0.903477,1.403089,7.201847,4.201847,not normal,13.243400,-0.416635
325,GENETEC,0104,TECHNOLOGY,TECHNOLOGY EQUIPMENT,ACE,2.550,2.560,1.59,0.040,1640000.0,...,0.020355,0.012213,0.074706,0.405062,3.225538,24.896898,21.896898,not normal,7.336216,3.501555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,YB-LA,5048LA,INDUSTRIAL PRODUCTS & SERVICES,BUILDING MATERIALS,BOND,0.000,0.045,0.00,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1049,YENHER,5300,CONSUMER PRODUCTS & SERVICES,AGRICULTURAL PRODUCTS,MAIN,0.000,0.815,0.00,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,YEWLEE,0248,INDUSTRIAL PRODUCTS & SERVICES,"INDUSTRIAL MATERIALS, COMPONENTS & EQUIPMENT",ACE,0.000,0.260,0.00,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,YONGTAI-PA,7066PA,PROPERTY,PROPERTY,MAIN,0.000,0.085,0.00,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Aggregate Data

In [30]:
sector_overview_df = merged_df.groupby("SECTOR").agg({f"BETA_{period}Y": "mean",
                                    f"INTERCEPT_{period}Y": "mean",
                                    f"annualized_excess_return_of_equity_{period}Y": "mean", 
                                    f"annualized_standard_deviation_of_equity_{period}Y": "mean"
                                    }).dropna().sort_values(f"INTERCEPT_{period}Y", ascending=False)
sector_overview_df

,BETA_5Y,INTERCEPT_5Y,annualized_excess_return_of_equity_5Y,annualized_standard_deviation_of_equity_5Y
SECTOR,,,,
ENERGY,1.283193,0.007593,-0.108229,10.058081
TECHNOLOGY,1.226589,0.005491,-0.062609,7.536826
HEALTH CARE,0.996701,0.003972,-0.233854,3.150862
INDUSTRIAL PRODUCTS & SERVICES,0.925520,0.002686,0.101575,4.093862
CONSUMER PRODUCTS & SERVICES,0.755166,0.002021,0.049354,3.550831
TELECOMMUNICATIONS & MEDIA,0.936721,0.001898,-0.055110,4.873592
TRANSPORTATION & LOGISTICS,0.869354,0.001827,0.123833,4.207143
PLANTATION,0.832601,0.001715,0.143020,2.610103
FINANCIAL SERVICES,0.828828,0.000983,0.059522,2.210793


In [31]:
sub_sector_overview_df = merged_df.groupby(["SUBSECTOR","SECTOR"]).agg({f"BETA_{period}Y": "mean",
                                    f"INTERCEPT_{period}Y": "mean",
                                    f"annualized_excess_return_of_equity_{period}Y": "mean", 
                                    f"annualized_standard_deviation_of_equity_{period}Y": "mean"
                                    }).dropna().sort_values(f"INTERCEPT_{period}Y", ascending=False)
sub_sector_overview_df

,,BETA_5Y,INTERCEPT_5Y,annualized_excess_return_of_equity_5Y,annualized_standard_deviation_of_equity_5Y
SUBSECTOR,SECTOR,,,,
OTHER ENERGY RESOURCES,ENERGY,1.951182,0.008628,-0.324205,5.201506
"ENERGY INFRASTRUCTURE, EQUIPMENT & SERVICES",ENERGY,1.164498,0.008400,-0.115976,11.300651
HEALTH CARE EQUIPMENT & SERVICES,HEALTH CARE,1.301352,0.006012,-0.533129,4.658945
TELECOMMUNICATIONS EQUIPMENT,TELECOMMUNICATIONS & MEDIA,0.908048,0.005978,-0.171908,7.776166
DIGITAL SERVICES,TECHNOLOGY,1.097519,0.005642,-0.029822,12.565282
TECHNOLOGY EQUIPMENT,TECHNOLOGY,1.308303,0.005616,-0.068653,5.304347
SOFTWARE,TECHNOLOGY,1.382853,0.005441,-0.186073,6.519702
SEMICONDUCTORS,TECHNOLOGY,0.968607,0.005159,0.126740,4.024013
CHEMICALS,INDUSTRIAL PRODUCTS & SERVICES,0.892765,0.004552,0.117943,3.849140


## Update To Google Spreadsheet

In [32]:
# Update data to google spreadsheet
import numpy

def update_worksheet(sheet_name:str, df: pandas.core.frame.DataFrame):
  workSheet = gs.worksheet(sheet_name)
  workSheet.clear()
  workSheet.append_row(df.reset_index().columns.values.tolist())
  workSheet.format("1:1", {"textFormat":{"bold":"True"}})
  workSheet.append_rows(df.reset_index().fillna("").values.tolist(), value_input_option='RAW', table_range ="A1")

update_worksheet("Bursa stock list", merged_df)
update_worksheet("Industry", sector_overview_df)
update_worksheet("Sub-Industry", sub_sector_overview_df)

## Chart Plotting

In [33]:
# Excess Return vs Standard Deviation

from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, PanTool, ZoomInTool, ZoomOutTool, WheelZoomTool, ResetTool, SaveTool
from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap

output_notebook()
# output_file("toolbar.html")

sector_list = ['TECHNOLOGY', 'CONSUMER PRODUCTS & SERVICES', 'PLANTATION',
      'EXCHANGE TRADED FUND-BOND', 'INDUSTRIAL PRODUCTS & SERVICES',
      'FINANCIAL SERVICES', 'PROPERTY', 'CONSTRUCTION', 'HEALTH CARE',
      '', 'TRANSPORTATION & LOGISTICS', 'ENERGY',
      'REAL ESTATE INVESTMENT TRUSTS', 'TELECOMMUNICATIONS & MEDIA', 'UTILITIES']
normalized: bool = True
filter_sector: bool= True

def normalize_data(df: pandas.core.frame.DataFrame, x:str, y:str):
  x = df[x]; y = df[y]
  df = df[x.between(x.quantile(0.05), x.quantile(0.95)) & y.between(y.quantile(0.05), y.quantile(0.95))]
  return df

def filter_value(df: pandas.core.frame.DataFrame, column: str, unique_val_in_col: list):
  return df[df[column].isin(unique_val_in_col)]

def plot_fig(df:pandas.core.frame.DataFrame, x:str, y:str):
  fig = figure(height=800,width=1000,tools="hover",  toolbar_location="above", 
              x_axis_label=x, y_axis_label=y,
              tooltips=[("STOCK","@STOCK_SYMBOL"),("SUBSECTOR", "@SUBSECTOR"), ("SECTOR", "@SECTOR"),
                        ("EXCESS RETURN", f"@annualized_excess_return_of_equity_{period}Y"),  
                        ("STANDARD DEVIATION", f"@annualized_standard_deviation_of_equity_{period}Y")])

  fig.add_tools(PanTool(), ZoomInTool(), ZoomOutTool(), WheelZoomTool(), ResetTool(), SaveTool())

  adjusted_df = df.copy(deep=True)

  if filter_sector:
    adjusted_df = filter_value(df, "SECTOR", sector_list)
  if normalized:
    adjusted_df = normalize_data(df,x=x,y=y)

  source = ColumnDataSource(data=adjusted_df)

  fig.scatter(source=source, y=y, x=x, legend_field="SECTOR", 
              fill_color=factor_cmap("SECTOR", Category20c[14], factors=adjusted_df["SECTOR"].unique()),
              line_color=None, size=10)

  fig.add_layout(fig.legend[0], 'right')
  return fig

show(plot_fig(df=merged_df, x = f"annualized_standard_deviation_of_equity_{period}Y", y=f"annualized_excess_return_of_equity_{period}Y"))